In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import zipfile
import os

zip_path = "/content/drive/MyDrive/shp.zip"
extract_path = "/content/shp_extracted"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

os.listdir(extract_path)


['shp']

In [ ]:
import os
import geopandas as gpd
import folium
from shapely.geometry import mapping
from IPython.display import display

# Folder where "shp" files are located
shp_folder = "/content/shp_extracted/shp"

# Function to recursively search for a .shp file
def find_shapefile(folder):
    for root, dirs, files in os.walk(folder):
        for file in files:
            if file.endswith(".shp"):
                return os.path.join(root, file)  # Return the first found .shp file
    return None

# Find the .shp file
shp_file_path = find_shapefile(shp_folder)

if shp_file_path is None:
    print("Shapefile not found!")
else:
    print(f"Shapefile found: {shp_file_path}")

    # Load data using GeoPandas
    gdf = gpd.read_file(shp_file_path)

    # Check coordinate reference system (CRS)
    if gdf.crs is None:
        print("CRS is missing! Setting to EPSG:4326 (WGS 84)")
        gdf.set_crs(epsg=4326, inplace=True)

    elif gdf.crs.to_epsg() != 4326:
        print(f"Current CRS: {gdf.crs}, converting to EPSG:4326")
        gdf = gdf.to_crs(epsg=4326)

    # Check if the dataset is empty
    if gdf.empty:
        print("WARNING: GeoDataFrame is empty!")
    else:
        # Get boundaries for correct visualization
        bounds = gdf.total_bounds  # [minx, miny, maxx, maxy]
        map_center = [(bounds[1] + bounds[3]) / 2, (bounds[0] + bounds[2]) / 2]  # [lat, lon]

        # Create an interactive map
        m = folium.Map(location=map_center, zoom_start=8)

        # Add a GeoJSON layer
        folium.GeoJson(gdf, name="Shapefile Layer").add_to(m)

        # Display the map
        display(m)


Shapefile found: /content/shp_extracted/shp/Germany/EMSR517/EMSR517_AOI17_GRA_PRODUCT_areaOfInterestA_r1_v2.shp
CRS is missing! Setting to EPSG:4326 (WGS 84)


In [ ]:
import os
import geopandas as gpd
import folium
from IPython.display import display

# Folder for Italy shapefiles
shp_folder = "/content/shp_extracted/shp/Italy"

# Function to recursively search for .shp files
def find_shapefile(folder):
    for root, dirs, files in os.walk(folder):
        for file in files:
            if file.endswith(".shp"):
                return os.path.join(root, file)  # Return the first found .shp file
    return None

# Search for the .shp file
shp_file_path = find_shapefile(shp_folder)

if shp_file_path is None:
    print("Shapefile not found for Italy!")
else:
    print(f"Shapefile found for Italy: {shp_file_path}")

    # Load data using GeoPandas
    gdf = gpd.read_file(shp_file_path)

    # Check the coordinate reference system (CRS)
    if gdf.crs is None:
        print("CRS is missing! Setting to EPSG:4326 (WGS 84)")
        gdf.set_crs(epsg=4326, inplace=True)  # Set to WGS 84

    elif gdf.crs.to_epsg() != 4326:
        print(f"Current CRS: {gdf.crs}, converting to EPSG:4326")
        gdf = gdf.to_crs(epsg=4326)  # Convert to WGS 84

    # Check if data is available
    if gdf.empty:
        print("WARNING: GeoDataFrame is empty!")
    else:
        # Determine bounds for correct display
        bounds = gdf.total_bounds  # [minx, miny, maxx, maxy]
        map_center = [(bounds[1] + bounds[3]) / 2, (bounds[0] + bounds[2]) / 2]  # [lat, lon]

        # Create an interactive map
        m = folium.Map(location=map_center, zoom_start=8)

        # Add a layer with geodata
        folium.GeoJson(gdf, name="Italy Shapefile Layer").add_to(m)

        # Display the map
        display(m)


Shapefile found for Italy: /content/shp_extracted/shp/Italy/EMSR468/EMSR468_AOI03_GRA_PRODUCT_naturalLandUseA_r1_v1.shp
CRS is missing! Setting to EPSG:4326 (WGS 84)


In [ ]:
import os
import geopandas as gpd
import pandas as pd
import folium
from IPython.display import display
from IPython.core.display import HTML

# Main folder containing the data
shp_folder = "/content/shp_extracted/shp"

# Function to search for .shp files by country
def find_shapefiles_by_country(folder, country):
    shp_files = []
    for root, dirs, files in os.walk(folder):
        for file in files:
            if file.endswith(".shp") and country.lower() in root.lower():
                shp_files.append(os.path.join(root, file))
    return shp_files

# Function to process and display .shp files
def process_shapefiles(shp_file_paths, country_name):
    if not shp_file_paths:
        print(f"No .shp files found for {country_name}!")
        return None

    print(f"Found {len(shp_file_paths)} .shp files for {country_name}:")
    for shp in shp_file_paths:
        print(f" - {shp}")

    # Read all .shp files into a list of GeoDataFrames
    gdfs = []
    for shp_file in shp_file_paths:
        gdf = gpd.read_file(shp_file)

        # Check CRS and convert to EPSG:4326
        if gdf.crs is None:
            print(f"CRS missing in {shp_file}! Setting to EPSG:4326 (WGS 84)")
            gdf.set_crs(epsg=4326, inplace=True)
        elif gdf.crs.to_epsg() != 4326:
            print(f"Current CRS ({gdf.crs}) in {shp_file}, converting to EPSG:4326")
            gdf = gdf.to_crs(epsg=4326)

        if not gdf.empty:
            gdfs.append(gdf)

    # Check if data exists
    if not gdfs:
        print(f"WARNING: All GeoDataFrames for {country_name} are empty!")
        return None

    # Merge all GeoDataFrames
    combined_gdf = gpd.GeoDataFrame(pd.concat(gdfs, ignore_index=True), crs="EPSG:4326")

    # Determine bounds for correct visualization
    bounds = combined_gdf.total_bounds  # [minx, miny, maxx, maxy]
    map_center = [(bounds[1] + bounds[3]) / 2, (bounds[0] + bounds[2]) / 2]  # [lat, lon]

    # Create an interactive map
    m = folium.Map(location=map_center, zoom_start=6)

    # Add a layer with the combined geodata
    folium.GeoJson(combined_gdf, name=f"{country_name} Shapefile Layers").add_to(m)

    # Display the map (fixed indentation)
    m.save(f"map_{country_name}.html")
    display(HTML(f"map_{country_name}.html"))


# Process data for Italy
italy_shp_files = find_shapefiles_by_country(shp_folder, "Italy")
process_shapefiles(italy_shp_files, "Italy")

# Process data for Germany
germany_shp_files = find_shapefiles_by_country(shp_folder, "Germany")
process_shapefiles(germany_shp_files, "Germany")


Found 327 .shp files for Italy:
 - /content/shp_extracted/shp/Italy/EMSR468/EMSR468_AOI03_GRA_PRODUCT_naturalLandUseA_r1_v1.shp
 - /content/shp_extracted/shp/Italy/EMSR468/EMSR468_AOI04_GRA_PRODUCT_naturalLandUseA_r1_v1.shp
 - /content/shp_extracted/shp/Italy/EMSR468/EMSR468_AOI06_GRA_PRODUCT_hydrographyA_r1_v1.shp
 - /content/shp_extracted/shp/Italy/EMSR468/EMSR468_AOI03_GRA_PRODUCT_transportationL_r1_v1.shp
 - /content/shp_extracted/shp/Italy/EMSR468/EMSR468_AOI03_GRA_PRODUCT_hydrographyA_r1_v1.shp
 - /content/shp_extracted/shp/Italy/EMSR468/EMSR468_AOI06_GRA_PRODUCT_hydrographyL_r1_v1.shp
 - /content/shp_extracted/shp/Italy/EMSR468/EMSR468_AOI06_GRA_PRODUCT_transportationL_r1_v1.shp
 - /content/shp_extracted/shp/Italy/EMSR468/EMSR468_AOI04_GRA_PRODUCT_hydrographyL_r1_v1.shp
 - /content/shp_extracted/shp/Italy/EMSR468/EMSR468_AOI04_GRA_PRODUCT_builtUpP_r1_v1.shp
 - /content/shp_extracted/shp/Italy/EMSR468/EMSR468_AOI04_GRA_PRODUCT_transportationL_r1_v1.shp
 - /content/shp_extracted/s